In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords# stopwords are those that add no meaning to the article, for eg: that this is etc
from nltk.stem.porter import PorterStemmer #gives root word of a particular word
from sklearn.feature_extraction.text import TfidfVectorizer# to covert text into feature vectors
from sklearn.model_selection import train_test_split
from  sklearn.linear_model import LogisticRegression
from  sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords of the language english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
#DATA PREPROCESSING/DATA CLEANING
news_dataset=pd.read_csv("/content/news.csv")

In [ ]:
news_dataset.shape

(6335, 4)

In [ ]:
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
#no. of missing values in dataset
news_dataset.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [ ]:
#reolacing all null values wil empty string
news_dataset=news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content']= news_dataset['title']+' '+news_dataset['text']


In [ ]:
print(news_dataset['content'])

0       You Can Smell Hillary’s Fear Daniel Greenfield...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathy U....
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [ ]:
#stemming
#example: actor, actress, acting --> act
stopWords=stopwords.words('english')
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stopWords=stopwords.words('english')
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopWords]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content']= news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])


0       smell hillari fear daniel greenfield shillman ...
1       watch exact moment paul ryan commit polit suic...
2       kerri go pari gestur sympathi u secretari stat...
3       berni support twitter erupt anger dnc tri warn...
4       battl new york primari matter primari day new ...
                              ...                        
6330    state depart say find email clinton specialist...
6331    p pb stand plutocrat pentagon p pb stand pluto...
6332    anti trump protest tool oligarchi inform anti ...
6333    ethiopia obama seek progress peac secur east a...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: content, Length: 6335, dtype: object


In [ ]:
#sepearting the data and label
X=news_dataset['content'].values
news_dataset['label'] = news_dataset['label'].map({'REAL': 0, 'FAKE': 1})

Y=news_dataset['label'].values

In [ ]:
print(X)

['smell hillari fear daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim sca

In [ ]:
print(Y)

[1 1 0 ... 1 0 0]


In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=50000,
    min_df=2,
    max_df=0.9
)
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2205243 stored elements and shape (6335, 50000)>
  Coords	Values
  (0, 35)	0.02426173949147292
  (0, 147)	0.020521227836931662
  (0, 149)	0.03083619983390287
  (0, 294)	0.04774632797675734
  (0, 361)	0.05305998395064742
  (0, 485)	0.01225848260221367
  (0, 630)	0.017925340820457665
  (0, 749)	0.0696717803384422
  (0, 768)	0.017474728178700567
  (0, 775)	0.016459631077460173
  (0, 792)	0.0392556664817235
  (0, 816)	0.013640491633861508
  (0, 857)	0.01682500530945728
  (0, 869)	0.032947147474616634
  (0, 1048)	0.023464045805154456
  (0, 1057)	0.018231623153968674
  (0, 1073)	0.03264713817957948
  (0, 1096)	0.013042304734728843
  (0, 1172)	0.025349605278779115
  (0, 1178)	0.03809784978852143
  (0, 1200)	0.015944131204676
  (0, 1406)	0.03433073360500262
  (0, 1487)	0.01981810286088949
  (0, 1584)	0.009390748466546134
  (0, 1698)	0.03359162536667696
  :	:
  (6334, 47176)	0.046672685066635314
  (6334, 47363)	0.012508299432891883
 

In [ ]:
# import numpy as np

# # Find indices where Y is not NaN
# non_nan_indices = ~np.isnan(Y)

# # Filter X and Y to remove NaN values
# X_filtered = X[non_nan_indices]
# Y_filtered = Y[non_nan_indices]

# #Splitting the dataset to training & test data

X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=42)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#training
model= LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
#EVALUATION- accuracy score
x_train_pred = model.predict(X_train)
training_data_accuracy= accuracy_score(x_train_pred,Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9607340173638517


In [ ]:
# accuracy score on test data
X_test_pred= model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_pred,Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9163378058405682


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score

# Predict labels
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(Y_test, y_pred)

# Precision
precision = precision_score(Y_test, y_pred)

# Recall
recall = recall_score(Y_test, y_pred)

# F1 Score
f1 = f1_score(Y_test, y_pred)

# Confusion Matrix
cm = confusion_matrix(Y_test, y_pred)

# Classification Report
report = classification_report(Y_test, y_pred)

# ROC AUC Score
probs = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(Y_test, probs)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("AUC Score:", auc)
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)


Accuracy: 0.9163378058405682
Precision: 0.8962406015037594
Recall: 0.9415481832543444
F1 Score: 0.9183359013867488
AUC Score: 0.9793931057853793

Confusion Matrix:
 [[565  69]
 [ 37 596]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.89      0.91       634
           1       0.90      0.94      0.92       633

    accuracy                           0.92      1267
   macro avg       0.92      0.92      0.92      1267
weighted avg       0.92      0.92      0.92      1267



In [ ]:
#to make a predictive system

X_new = X_test[23]

prediction= model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(Y_test[3])

1


In [ ]:
#add graphs
#learn to speak
#modify ppt